In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from pymongo import MongoClient

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
stations = client.bda.stations
weather = client.bda.weather

In [ ]:
def get_collection(collection):
    documents = []
    cursor = collection.find({})
    for document in cursor:
        documents.append(document)
    return documents

In [ ]:
stations_df = pd.DataFrame(get_collection(stations))
stations_df = stations_df.rename(columns={"STATION_ID": "station_id"})

In [ ]:
weather_df = pd.DataFrame(get_collection(weather))

In [ ]:
weather_stations = pd.merge(weather_df,stations_df, on="station_id", how="left")
weather_stations

In [ ]:
def get_temp_between_dates(data, date1, date2):
    result = data.loc[data['date']>date1]
    result = result.loc[result['date']<date2]
    result = result.astype({'date': 'string'})
    return result

In [ ]:
plot_data = get_temp_between_dates(weather_stations, '2020-08-20 00:00:00', '2020-08-31 00:00:00')
plot_data

In [ ]:
fig = px.scatter_mapbox(
    plot_data, lat="LAT", lon="LON", size_max=50,
    color="temperature", color_continuous_scale=px.colors.sequential.Pinkyl,
    hover_name="STATION_NAME", 
    hover_data=["station_id", "temperature", "STATE", "STATION_HEIGHT"],
    mapbox_style="open-street-map",
    zoom=4,
    height=500,
    width =500,
    animation_frame="date", animation_group="station_id",
    title = "Temperatur in Deutschland"
)

In [ ]:
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 50
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 50
fig.layout.coloraxis.showscale = True
fig.layout.coloraxis.colorbar.len = 0.8
fig.layout.coloraxis.cmin = -10
fig.layout.coloraxis.cmax = 40
fig.layout.sliders[0].pad.t = 10
fig.layout.updatemenus[0].pad.t= 10

In [ ]:
fig.show()